In [15]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 22 17:12:01 2021

@author: User
"""
# import 爬蟲所需要的東西
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import os,sys,time
from PIL import Image
from urllib import request
import requests
import json
import sys
import modin.pandas as pd
from iteration_utilities import duplicates
import asyncio
import unittest
from numba import jit
import csv


# P.S.
# python 預設調用  print， 會調用 __str__ 函數。
# __rerp__ 是預覽



# 實作　Node 單位


class Node:
    #建構子，初始化節點
    #初始authority跟 hub,值皆為1
    def __init__(self, ID): #一定會有self這個值，並用self代稱上面的class_name(Node)
        self.ID = ID
        self.parents = []
        self.children = []  #初始的children 跟 parents串列裡面都是空的。(因為都沒有建立任何連結)
        self.auth = 1.0
        self.hub = 1.0
    # __str__ 是print會調用的函數
    def __str__(self):
        return str(self.ID)
        
    def show(self): #所有的function裡面都要有self，代指使用自己方法的本身。
        print(f"This's {self.ID}")
        
    #link_child 跟 link_parent，為網站間的有向連結，但其並沒有考慮到各child節點或各parent節點之間加入到"children"或"parent"串列的先後順序。
    #只能計算此節點所有連出的節點個數計算。
        
    def link_child(self,new_child): #這Method中有兩個參數，self如上述，"new child" 為其他的Node類物件，近來這個程式的代稱為"new_child"
        for child in self.children: #在呼叫Method link_child的 Node 中，依序搜尋children 串列。
            if(new_child.ID == child.ID): #若已經有了，回傳None
                print("duplicate") #若有重複印出"duplicate"
                #print(child.ID)
                return None
        self.children.append(new_child)
        
    def link_parent(self,new_parent): #同上
        for parent in self.parents:
            if (new_parent.ID == self.ID):
                print("duplicate")
                #print(self.ID)
                return None
        self.parents.append(new_parent)
        
    def show_children(self): #印出一個節點的所有孩子(印出這個node所有連出的連結)
        if len(self.children) == 0:
            print("this node :",self.ID,", children_list is empty.")
        else:
            for i in range(len(self.children)):
                print(self.children[i].ID)
        
    def show_parents(self): #印出一個節點的所有parent(印出這個node所有連入的連結)
        for i in range(len(self.parents)):
            print(self.parents[i].ID)
            
    def update_hub(self):  #更新此點的hub分數
        hub_score = 1
        for i in self.children:
            hub_score += i.hub
        return hub_score
    
    def update_auth(self):   #更新此點的authority分數
        authority_score = 0
        for i in self.parents:
            authority_score += i.auth
        return authority_score
    
    def show_indegree(self):  #算出所有此點的 in_degree
        pass
    
    def show_outdegree(self):  #算出所有此點的 out_degree
        pass
    
    def normalize_hub_score(self):   #將此點的hub分數除以全部的hub分數總合
        pass
    
    def normalize_authority_socre(self):   #將此點的authority分數除已全部的authority分數總合。
        pass
    
#實作 Graph

class Graph:   #在這個網絡上(圖)的所有網站(點)集合
    def __init__(self, G_name):
        self.name = G_name  #圖的名字
        self.nodes = [] #一開始的nodes集合為空
        self.edges = [] #所有的邊集合我也設一個串列存起來，但這邊我目前還沒有想到怎麼加入可以加入有向邊。
    
    def insert_node(self, node):  #將外來的點加入這個圖的點集合中
        try:    
            self.nodes.append(node)
            # print("successfully insert！")
        except error as e:
            print(f"insert fail:{e}")
    
    def display_hub_authority_score(self):
        pass
    
    def find_node(self, ID):  # 要找的點的名字，也就是網址
        for node in self.nodes: # 在自己圖中的節點做尋找
            if node.ID == ID:
                # print("found")
                return node # 找到的話就回傳節點
        else: # 否則就說沒找到
            # print("Not found")
            return None # 回傳空值

# 將所有 ID 一樣的都刪除。
def set_node_list(list_a):
    # 比較所有的ID名稱，先用for迴圈將所有的ID名稱存起來。
    all_name = [] #所有名字的列表
    for i in list_a:
        all_name.append(i.ID)
    all_name_count = [] #計算每個網站重複的數量
    for index,i in enumerate(list_a):  #對每個list_a 的元素做檢查
        count = 0
        for k in all_name: # 在所有的名字中搜尋，如果有一樣的名字，count就加一。
            if i.ID == k:
                count +=1
        all_name_count.append(count)
    for index,i in enumerate(all_name_count):  #超過一次以上的就刪除
        if i > 1:
            del list_a[index]
            del all_name[index]
            del all_name_count[index]
    return list_a                   #回傳一list 其為不重複的 node版本

def search_all_link(query_link):
    a = []  # 這個網站所有的 outlink
    html = requests.get(query_link, timeout = 10)
    # 為防止爬蟲卡死情況(request 不會自行斷開)，故上面設置 timeout 參數，設定固定時間若未完成則 斷開連結。
    html.encoding = 'UTF-8' #使用html編碼。
    sp = BeautifulSoup(html.text, 'html5lib') #指定'html5lib'作為解析器
    #若只使用sp.link 則只會找出第一筆
    #故這邊要使用"find.all"語法
    link_search = sp.find_all("a") #找出所有的 <a> (超連結)
    for i in range(len(link_search)):
        k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
        if k[0:5] == "https": #只留下"https"
            a.append(k)
    return a  #回傳一列網址(type list)

#先將所有node_list中的資料轉換成type str存入到list再寫入檔案
def node_to_text(all_node_list):
    node_list_txt = [] # node_list 的所有內容
    for i in all_node_list:
        tmp_list = [] # 初始化
        # 依序記錄結構化的node的每個資料
        tmp_list.append(i.ID)
        # 這邊嘗試寫遞迴函式，不停往下找
        # 加入cildren
        tmp_children = []
        for j in i.children:
            tmp_children.append(j.ID)
        tmp_list.append(tmp_children)
        # 加入parents
        tmp_parents = []
        for k in i.parents:
            tmp_parents.append(k.ID)
        tmp_list.append(tmp_parents)
        tmp_list.append(i.auth)
        tmp_list.append(i.hub)
        # 再加入list
        node_list_txt.append(tmp_list)
    return node_list_txt  # 回傳一個 list
def listDups(tmp_list_aaa): # 記錄重複的功能
    return list(duplicates(tmp_list_aaa))
def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    flag_head = False # 一開始起頭開關為False
    tmp_word_list = [] # 裝暫存要轉變成文字的網址，每次只會存一個網址，之後就會被清空
    tmp_url_list = [] # 裝整理好的網址
    for url in bbb: # 在這bbb串列中做遞迴
        for char in url:
            if char == "'": # 當遇見"單引號"時switch (此時Flag_head只有兩種狀態)
                flag_head = not(flag_head) # 轉換開關
                # tmp_word_list.append(char) # 將其收錄至暫存的字串列。
            if flag_head == True:
                tmp_word_list.append(char) # 因為只有h會將其打開，所以一路收錄char 至 tmp_word_list
            if flag_head == False:
                StrA = "".join(tmp_word_list)
                StrA = StrA[1:]  # 去掉前端的單引號。 
                if len(tmp_word_list) != 0: # 不為空才加入
                    tmp_url_list.append(StrA)
                tmp_word_list = [] # 清空tmp_word_list
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址

# 111/1/1發現這邊有問題，雖然建立的nodes名字都一樣，但是因為是從不同檔案load進來的，會有問題。
# 111/1/1目前解決方法，隨然load進來的是不同檔案，但是會load到同個圖中。
def load_node_from_csv_out_side(start_num, end_num, df_name): # 讀csv檔的資料 # 並且加入子節點 # 也同時加入父節點
    # 參數分別為，load加入的檔案的第幾筆到第幾筆，讀入黨案名稱。
# 每個點依序建立連結
    # 在這邊才讀取檔案是為了要避免讀取全域變數
    df = pd.read_csv(df_name)
    start_time = time.time()  #開始執行時間
    for i in range(start_num, end_num + 1): # 先存1-9000筆
        bbb = df["2"][i] # 將一整串str去頭去尾。
        url_list = str_to_url(bbb)  # 會回傳一個整理好的串列
        tmp2 = Graph_all.nodes[i] # 選取第幾個node
        for j in url_list: # 對找到的每個網站
            # 要先觀察 "j" 這個網站名字，是不是已經存在在原有的193721筆資料中。
            a = Graph_all.find_node(j)
            if a != None: # 有找到值，代表此網站已經在圖中了。此時 a 就已經是圖中的節點。 type == node
                tmp2.link_child(a)
                a.link_parent(tmp2)
            else: # 沒找到值，代表這是新的點
                tmp_node = Node(j) # 先將網站節點化
                Graph_all.insert_node(tmp_node) # 圖中也要加入這個新的點。
                Graph_all.nodes[i].link_child(tmp_node) # 加入node化的子節點。 # 這邊確認沒有重複。
                tmp_node.link_parent(tmp2) # 連結原節點。
    print("花費:" + str(float(time.time() - start_time)) + "秒")
    print("finish")
def Diff(li1, li2): 
    return (list(set(li1).symmetric_difference(set(li2))))  
# 判別兩列表不同元素
def diff(list_a, list_b):
    list_1 = list_a.copy()
    list_2 = list_b.copy()
    intersection = [x for x in list_1 for y in list_2 if x == y]
    return intersection

def load_out_link(start, end, csv_file_name): # 讀所有out_link # 回傳一列表
    
    df = pd.read_csv(csv_file_name)
    return_list = [] # 要回傳的列表
    for index in range(start,end+1): # 將前9000筆資料加入列表中
        bbb = df["2"][index]
        url_list = str_to_url(bbb)
        for j in url_list:
            if j[0:4] == "http": # 等於http的才加入。
                #print(j) # 撿查
                return_list.append(j) # 依序加入
    return return_list
    
# 實行二分搜尋
# 定義：在排序好的數列裡，找某 target，找到就回傳其 index，否則回傳 -1
def binary_search(data_list, key_data):  #要加入的圖的點的列表(已知且已排序), 要尋找的 資料
    #設置選取範圍的指標
    low = 0
    upper = len(data_list) - 1
    while low <= upper:
        mid = int((low + upper) / 2) #取中間索引的值
        if data_list[mid] < key_data:    #若搜尋值比中間的值大，將中間索引+1，取右半
            low = mid + 1
        elif data_list[mid] > key_data:  #若搜尋值比中間的值小，將中間索引+1，取左半
            upper = mid - 1
        else:                    #若搜尋值等於中間的值，則回傳
            return mid # 回傳的是索引值。
    return "Node Not Found!"


#  這邊一定要針對外部的圖進行修改，無法不用全域變數。
#  這邊其實已經解決先後順序的問題了，所以可以嘗試看看非同步的寫法。
#  此函式適用於所有load進來的檔案中已經沒有新node(所有的不重複網站名稱都已經存在在以建立之圖中)
#  之後應該要建立一另外之函式for 新加入的網站(以上述的csv檔案儲存)
def build_module_all_nodes_found(original_url, graph, out_link_list_http): 
    # 確定全部的點都加入了才可以使用此函式
    # 參數 原始的網站，要加入的圖(裡面的點已排序)(對這個圖進行操作)，out_link列表
        # 找到原始網站的那個點
        # 0.22s
        # 這邊改用 binary search
    All_graph_nodes_list = graph.nodes # 使用那個圖的點 #  指的是同一個位置 # 這個圖已經排序了
    key_1 = binary_search(All_graph_nodes_list, original_url)
    node_tmp = All_graph_nodes_list[key_1] 
   # print("網站",node_tmp.ID)
    for out_url in out_link_list_http:
        if len(out_link_list_http) != 0:
            key_2 = binary_search(All_graph_nodes_list, out_url)
            # 0.22s
            if key_2 != "Node Not Found!":
                # 利用指標操作找到那個點
                node_tmp_2 = All_graph_nodes_list[key_2] 
                node_tmp.link_child(node_tmp_2)
                node_tmp_2.link_parent(node_tmp)
             #   print("out_link", node_tmp_2.ID)
        else:
            print("empty_outlink")

In [2]:
# 1110116 再更新想法，先不要創node ，因為檢索物件屬性這階段比較慢，所以先將所有的190000多筆node名字存入一個list
# 再迭代每個網址的children，利用：
# def Diff(li1, li2): 
#    return (list(set(li1).symmetric_difference(set(li2))))
# 先快速比對兩創列不同之處，其不同之處就是要新創的node
# 全部比對完之後，應該可以創出一大於190000多筆資料的總node數字
# 以及因為每個children串列都用set整理過，理論上應該會大大降低讀取時間。

In [3]:
# 加入所有的網站(包含original_url跟out_link)

In [4]:
# 初始化由此開始

total_url = []
path = "url_list.txt"
f = open(path, "r",encoding ="utf-8")
level_0 = f.readlines()
for i in level_0:
    total_url.append(i[:-1])

total_url = list(set(total_url))
print(len(total_url))
counter=0

353


In [5]:
csv.field_size_limit(500 * 1024 * 1024)
for i in range(9): # 有幾個檔案range裡面就放幾
    this_level_total_url_list = []
    counter = i
    file_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv' 
    with open(file_name, newline='', encoding = "utf-8") as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
        for row in rows:
            if len(row) == 0: # 空行跳過
                continue
            if row[0][0:4] == "http":
                this_level_total_url_list.append(row[0]) # 原始url也要加進去
            children_list = str_to_url(row[2]) # 每個url的out_link
            for url in children_list:
                if url[0:4] == "http":
                    this_level_total_url_list.append(url)
        total_url += this_level_total_url_list
        total_url = list(set(total_url))
        print(len(total_url)) # 已經是不重複的了

11744
83821
445237
679822
849136
1110631
1350738
1530606
1754370


In [6]:
# 再次確認
print("不重複總網站數目", len(total_url))
total_url[:10] # 觀察前10筆

不重複總網站數目 1754370


['https://tw.video.search.yahoo.com/search/video;_ylt=AwrtFGN0jPthDRkAlAZr1gt.;_ylu=Y29sbwN0dzEEcG9zAzIEdnRpZAMEc2VjA3Nj?p=%E5%88%AE%E5%88%AE%E6%A8%82%E4%B8%AD%E7%8D%8E%E7%A7%98%E8%A8%A3&ei=UTF-8&turl=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOVP.4LwElOsUauoFPlj9EFUE7wEsDg%26pid%3DApi%26w%3D148%26h%3D78%26c%3D7&rurl=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DDzAAv0t5N5E&tit=%E5%AF%A6%E6%B8%AC%E4%BA%94%E7%A8%AE%E5%88%AE%E5%88%AE%E6%A8%82%E5%BF%85%E4%B8%AD%E7%8D%8E%E7%9A%84%E7%A7%98%E7%AC%88%EF%BC%81%E6%98%AF%E9%83%BD%E5%B8%82%E5%82%B3%E8%AA%AA%E9%82%84%E6%98%AF%E7%9C%9F%E7%9A%84%E6%9C%89%E6%95%88%EF%BC%81%EF%BC%9F&pos=4&vid=83d9805c2d51bea4676d1285660c7f79&sigr=QE.gnODFZeXU&sigt=2pJV4gpZYg06&sigi=UbNzvH_6jSP1',
 'https://help.openstreetmap.org/users/20620/msiipola',
 'https://stats.moe.gov.tw/enchartweb/',
 'https://www.stthomas.edu/officeofgc/universitypolicies/',
 'https://bibblemcgee.tumblr.com/post/675178405453545472',
 'https://hu.usembassy.gov/category/gender-issues/',
 'https:/

In [7]:
# 排序 以便進行binary search 演算法
Sorted_All_list = sorted(total_url)
# 一樣檢查前10筆
Sorted_All_list[:10]

['http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share',
 'http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share&via=ucsc',
 'http://webdam.inria.fr/Jorge/files/wdm-yago.pdf" ',
 'http://www.facebook.com',
 'http://www.google.com/books%3Fid%3DSMdiDwAAQBAJ%22%7D&client=17&contact_type=anno&source=partner-google-print',
 'http://www.google.com/books%3Fid%3DsLJXV_z8XC4C%22%7D&client=17&contact_type=anno&source=partner-google-print',
 'http://www.nap.edu/download.php?record_id=13564',
 'http://www.thecanadianencyclopedia.ca/en/article/physics',
 'https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-ginger-750ml',
 'https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-grand-poppy-amaro-liqueur-750ml']

In [8]:
# 建立所有點 塞入圖中

start_time = time.time()  #開始執行時間
Graph_all = Graph("Authority_&_Hub") # 圖結構
print(Graph_all.name) # 印出圖名
for index, i in enumerate(Sorted_All_list): # 將其節點化，並加入圖中
    tmp = Node(i)
    Graph_all.insert_node(tmp) # 所以這邊會依序加入，之後可以利用這個特徵操控
print(len(Graph_all.nodes))



Authority_&_Hub
1754370


In [11]:
for node in Graph_all.nodes[:10]:
    print(node.ID)

http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share
http://news.ucsc.edu/2022/01/covid-reminders.html?ref=share&via=ucsc
http://webdam.inria.fr/Jorge/files/wdm-yago.pdf" 
http://www.facebook.com
http://www.google.com/books%3Fid%3DSMdiDwAAQBAJ%22%7D&client=17&contact_type=anno&source=partner-google-print
http://www.google.com/books%3Fid%3DsLJXV_z8XC4C%22%7D&client=17&contact_type=anno&source=partner-google-print
http://www.nap.edu/download.php?record_id=13564
http://www.thecanadianencyclopedia.ca/en/article/physics
https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-ginger-750ml
https%3A%2F%2Fbottlebarn.com%2Fproducts%2Ffruitlab-grand-poppy-amaro-liqueur-750ml


In [25]:
# 在做binary_search時，需要一個所有"名字的列表"(就是上面做出的Sorted_All_list)
# 而且因為Sorted_All_list 跟 Graph_all，長度一樣，所以可以利用這個特性使用相同指標操作。
start_time = time.time() 
for i in range(9):
    counter = 0
    file_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv'
    with open(file_name, newline='', encoding = "utf-8") as csvfile:
        rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
        for row in rows:
            if len(row) == 0: # 空行跳過
                continue
            if row[0][0:4] == "http":
                # 定位要加入的original_url
                index = binary_search(Sorted_All_list, row[0]) # 這邊找出的index 值，會跟Graph_all.nodes的index值一樣。
                if index != "Node Not Found!": #雖然理論上這行會執行，但是還是嚴謹點，以防漏網之魚
                    tmp_node = Graph_all.nodes[index] 
                else:
                    print("Node Not Found", row[0])
            children_list = str_to_url(row[2]) # 每個url的out_link
            for url in children_list:
                if url[0:4] == "http":
                    index_2 = binary_search(Sorted_All_list, url)
                    if index_2 != "Node Not Found!":
                        tmp_node_2 = Graph_all.nodes[index_2] 
                        tmp_node.link_child(tmp_node_2)
                        tmp_node_2.link_parent(tmp_node)
                    else:
                        print("Node Not Found", url)
    counter += 1
print("花費:" + str(float(time.time() - start_time)) + "秒")

duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


...

duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate
duplicate


In [27]:
# 檢查是否有正確加入
test = Graph_all.find_node("https://tw.buy.yahoo.com/search/product?p=%E5%A5%B3%E5%A4%96%E5%A5%97")

In [31]:
for node in test.parents:
    print(node.ID)

https://tw.buy.yahoo.com/act/oneec/freeshipping.html#sto991
https://tw.buy.yahoo.com/act/oneec/newaffair.html
https://tw.buy.yahoo.com/act/oneec/newaffair.html#auc


In [33]:
import csv
# 注意！目前存入csv檔案中的圖，其節點排序會跟原本不一樣，是無序的，但不影響之後運算。
# 將目前所作的圖，暫存到一個csv檔案中，以便後續利用。
def load_Garph_to_csv(Graph,Graph_file_name): # 參數： 圖, 輸出檔案名
    # 參數說明： Graph: 要加入的圖、從第幾個點開始、到第幾點結束。
    Graph_nodes = []
    count = 0
    for node in Graph.nodes:
        every_node = []
        every_node.append(count) # 照順序加入
        every_node.append(node.ID)
        tmp_parent = []
        for url in node.parents:
            tmp_parent.append(url.ID)
        every_node.append(tmp_parent)     # list格式
        tmp_children = []
        for url in node.children:
            tmp_children.append(url.ID)
        every_node.append(tmp_children)  # list格式
        every_node.append(node.auth)  # int
        every_node.append(node.hub)   # int
        #上面加完每點的各元素。
        count += 1
        Graph_nodes.append(every_node) # 在將每個點加入圖點總列表中
        #print("Graph_node append successfully")
    # 至此為止應該會將四個list都完整加入完成
    with open(Graph_file_name, 'w', newline='', encoding='UTF-8') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_ALL,delimiter=';')
        writer.writerows(Graph_nodes)

In [34]:
start_time = time.time()
load_Garph_to_csv(Graph_all, "Graph0205.csv")
print("花費:" + str(float(time.time() - start_time)) + "秒")

花費:20.2725088596344秒


In [101]:
len(Graph_all.nodes)

2172694

In [ ]:
# 以下為測試

In [ ]:
# HITS 演算法的四個步驟
# 1. 

In [102]:
def HITS_one_iter(graph):
    node_list = graph.nodes

    for node in node_list:
        node.update_auth()
    for node in node_list:
        node.update_hub()

    graph.normalize_auth_hub()

In [82]:
# 現在先檢查圖有沒有被加成功
# 先看原始圖
for i in Graph_all.nodes[:1000]:
    print(i.ID)
    print(i.children)
# 再看增加過點後的圖。



0
[]
http://cdn.omninoggin.com/wp-includes/images/smilies/icon_sad.gif
[]
http://cdn.omninoggin.com/wp-includes/images/smilies/icon_sad.gif\
[]
http://planetquest.jpl.nasa.gov/system/interactable/5/index.html
[]
http://www.thecanadianencyclopedia.ca/en/article/camillien-houde
[]
http://www.thecanadianencyclopedia.ca/en/article/media-convergence
[]
https%3A%2F%2Flavisionweb.com%2F2020%2Fcoronavirus%2Fel-poder-de-la-confianza-y-la-verdad%2F
[]
https%3A//www.youtube.com/embed/z-0a9PqN7Rs%3Fwmode%3Dopaque
[]
https//www.fbi.gov/contact-us/fbi-headquarters/the-fbi-experience
[]
https//www.ucalgary.ca/pubs/calendar/
[]
https://
[]
https:// networkcontagion.us/reports/covid-19-conspiracy-and-contagious-sedition-a-case-study-on-the-militia- sphere/
[]
https://#
[]
https://%20www.worldbank.org/lac
[]
https://.td.org/research-reports/2017-state-of-the-industry
[]
https:////employsure.com.au/contact/
[]
https:////www.usa.gov/
[]
https:///bmczool.biomedcentral.com/articles/10.1186/s40850-020-00055-

https://9foundations.forhealth.org/
[]
https://9to5mac.com/2021/05/01/idc-apple-tablet-market-ipad/
[]
https://9to5mac.com/2021/09/16/aapl-shareholder-resolution-repair/
[]
https://AcademiaBID.iadb.org
[]
https://AirPassengerProtection.ca
[]
https://Aka.ms/AzureSpace
[]
https://Bit.ly/HelloIDNTimes
[]
https://COVID19.CA.gov
[<__main__.Node object at 0x0000023CD7994B80>, <__main__.Node object at 0x0000023CD7994B20>, <__main__.Node object at 0x0000023CEC7E1190>, <__main__.Node object at 0x0000023CDFA26FA0>, <__main__.Node object at 0x0000023CF4521AF0>, <__main__.Node object at 0x0000023CEE880D00>, <__main__.Node object at 0x0000023CD799A7C0>, <__main__.Node object at 0x0000023CF4881AC0>, <__main__.Node object at 0x0000023CDEB2F0A0>, <__main__.Node object at 0x0000023CDC8D01C0>, <__main__.Node object at 0x0000023CF3BDCD00>, <__main__.Node object at 0x0000023CDBCF1CD0>, <__main__.Node object at 0x0000023CCF8FD2E0>, <__main__.Node object at 0x0000023CEC7DE9D0>, <__main__.Node object at 0x00